In [1]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),
                             minval = low, maxval = high,
                             dtype = tf.float32)

In [3]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.softplus,
                optimizer = tf.train.AdamOptimizer(), scale = 0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(
                        self.x + scale * tf.random_normal((n_input,)),
                        self.weights['w1']),self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden,
                                              self.weights['w2']),
                                    self.weights['b2'])
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(
                                        self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype= tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,self.n_input], 
                                                 dtype= tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype= tf.float32))
        return all_weights
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer),
                                 feed_dict = {self.x: X, self.scale: self.training_scale})
        return cost
    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X,
                                                    self.scale: self.training_scale})
    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x: X,
                                                      self.scale: slef.training_scale})
    def generate(self, hidden = None):
        if hidden is None:
            hidden = np.random.normal(size = self.weights["b1"])
        return self.sess.run(self.reconstruction,
                            feed_dict = {self.hidden : hidden})
    
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x: X,
                                                              self.scale: self.training_scale
                                                              })
    def getWeights(self):
        return self.sess.run(self.Weights['w1'])
    
    def getBiases(self):
        return self.sess.run(self.Weights['b1'])
    
        

In [4]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) -  batch_size)
    return data[start_index: (start_index + batch_size)]
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input = 784,
                                              n_hidden = 200,
                                              transfer_function = tf.nn.softplus,
                                              optimizer = tf.train.AdamOptimizer(learning_rate = 0.001),
                                              scale = 0.01)
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
    if epoch % display_step == 0:
        print("Epoch:",'%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
print("Total cost:" + str(autoencoder.calc_total_cost(X_test)))

Epoch: 0001 cost= 19652.754695455
Epoch: 0002 cost= 12699.005975000
Epoch: 0003 cost= 11226.952693182
Epoch: 0004 cost= 10526.979258523
Epoch: 0005 cost= 10636.435627841
Epoch: 0006 cost= 9097.886340341
Epoch: 0007 cost= 9785.000426705
Epoch: 0008 cost= 8160.794692045
Epoch: 0009 cost= 8483.496272727
Epoch: 0010 cost= 7966.605986932
Epoch: 0011 cost= 8196.456972159
Epoch: 0012 cost= 8160.348203409
Epoch: 0013 cost= 8928.716750000
Epoch: 0014 cost= 7616.621878409
Epoch: 0015 cost= 8674.605702273
Epoch: 0016 cost= 8912.537676705
Epoch: 0017 cost= 7686.117057386
Epoch: 0018 cost= 8888.041581250
Epoch: 0019 cost= 8390.109765341
Epoch: 0020 cost= 7574.932049432
Total cost:649477.0


In [6]:
# Multi-layer 
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
in_units = 784
h1_units = 300
w1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))
b1 = tf.Variable(tf.zeros([h1_units]))
w2 = tf.Variable(tf.zeros([h1_units, 10]))
b2 = tf.Variable(tf.zeros([10]))

In [11]:
x = tf.placeholder(tf.float32, [None, in_units])
keep_prob = tf.placeholder(tf.float32)
hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, w2) + b2)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),
                                             reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0, 3).minimize(cross_entropy)

In [12]:
tf.global_variables_initializer().run()
for i in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys, keep_prob: 0.75})

In [13]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels,
                    keep_prob: 1.0}))

0.098
